In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [29]:
total = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("total shape : {}, test shape : {}".format(total.shape, test.shape))

total shape : (368088, 12), test shape : (15120, 12)


In [30]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [31]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

# "year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"
features = ["month", "hour", "구분_int", "기온(°C)"]
# features = ["year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"]
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

X = total[features]
y = total["공급량"]

In [32]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)
print(test.shape)

(306768, 4) (306768,)
(61320, 4) (61320,)
(15120, 12)


In [33]:
model = Sequential()
model.add(Dense(units = 16, input_dim = 4, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 16)                80        
                                                                 
 dense_13 (Dense)            (None, 32)                544       
                                                                 
 dense_14 (Dense)            (None, 64)                2112      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                                 
 dense_17 (Dense)            (None, 1)                 17        
                                                                 
Total params: 5,361
Trainable params: 5,361
Non-traina

In [34]:
model.compile(loss = "mean_squared_error", 
              optimizer = "rmsprop", 
              metrics = ["accuracy"])

early_stopping = EarlyStopping(patience = 30)

hist = model.fit(train_x, train_y, 
                 epochs = 200, 
                 batch_size = 100, 
                 validation_data = (val_x, val_y), 
                 callbacks = [early_stopping])

Epoch 1/200
3068/3068 [==============================] - 4s 960us/step - loss: 576803.8750 - accuracy: 0.0000e+00 - val_loss: 630334.9375 - val_accuracy: 0.0000e+00
Epoch 2/200
3068/3068 [==============================] - 3s 876us/step - loss: 403858.8438 - accuracy: 0.0000e+00 - val_loss: 420579.0312 - val_accuracy: 0.0000e+00
Epoch 3/200
3068/3068 [==============================] - 3s 1ms/step - loss: 151107.4844 - accuracy: 0.0000e+00 - val_loss: 91485.4219 - val_accuracy: 0.0000e+00
Epoch 4/200
3068/3068 [==============================] - 4s 1ms/step - loss: 80820.7578 - accuracy: 0.0000e+00 - val_loss: 392308.4375 - val_accuracy: 0.0000e+00
Epoch 5/200
3068/3068 [==============================] - 4s 1ms/step - loss: 59325.8867 - accuracy: 0.0000e+00 - val_loss: 68619.8281 - val_accuracy: 0.0000e+00
Epoch 6/200
3068/3068 [==============================] - 5s 2ms/step - loss: 47593.8398 - accuracy: 0.0000e+00 - val_loss: 40971.3750 - val_accuracy: 0.0000e+00
Epoch 7/200
3068/3068 [=

3068/3068 [==============================] - 3s 873us/step - loss: 22383.8203 - accuracy: 0.0000e+00 - val_loss: 47844.3555 - val_accuracy: 0.0000e+00
Epoch 52/200
3068/3068 [==============================] - 3s 926us/step - loss: 22343.0586 - accuracy: 0.0000e+00 - val_loss: 26515.2344 - val_accuracy: 0.0000e+00
Epoch 53/200
3068/3068 [==============================] - 3s 874us/step - loss: 22338.1094 - accuracy: 0.0000e+00 - val_loss: 31145.0410 - val_accuracy: 0.0000e+00
Epoch 54/200
3068/3068 [==============================] - 3s 932us/step - loss: 22399.3477 - accuracy: 0.0000e+00 - val_loss: 46869.4258 - val_accuracy: 0.0000e+00
Epoch 55/200
3068/3068 [==============================] - 3s 992us/step - loss: 22269.3262 - accuracy: 0.0000e+00 - val_loss: 39652.2305 - val_accuracy: 0.0000e+00
Epoch 56/200
3068/3068 [==============================] - 3s 978us/step - loss: 22286.9004 - accuracy: 0.0000e+00 - val_loss: 28244.1562 - val_accuracy: 0.0000e+00
Epoch 57/200
3068/3068 [=====

In [35]:
model.evaluate(val_x, val_y)

1917/1917 [==============================] - 1s 636us/step - loss: 32716.4863 - accuracy: 0.0000e+00


[32716.486328125, 0.0]

In [36]:
pred = model.predict(val_x)

true_y = pd.DataFrame({"y" : val_y})
pred_y = pd.DataFrame(pred)

true_y.reset_index(drop = True, inplace = True)
true_y.reset_index(inplace = True)
pred_y.reset_index(inplace = True)

print("NMAE : {:.6f}".format(nmae(true_y, pred_y)))

NMAE : 0.528145


In [24]:
pred_y

,index,0
0,0,2071.359375
1,1,1840.957764
2,2,1671.306763
3,3,1761.066162
4,4,1912.756104
...,...,...
61315,61315,629.390991
61316,61316,608.711670
61317,61317,592.297058
61318,61318,558.401062


In [25]:
true_y

,index,y
0,0,1765.008
1,1,1679.186
2,2,1610.885
3,3,1604.123
4,4,1711.506
...,...,...
61315,61315,681.033
61316,61316,669.961
61317,61317,657.941
61318,61318,610.953


In [26]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [27]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/19_4_features_DeepLearning_model.csv", index = False)